## 시작

In [ ]:
# ------ LIBRARY -------#
import numpy as np
import os
import pickle
import sys
import pandas as pd
import re
import cv2
import json
# torch
import torch
import torch.cuda.amp as amp
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
from glob import glob

import torch.nn as nn
import torch.nn.functional as F

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau, MultiStepLR, OneCycleLR

import math
import torch
from torch.optim.optimizer import Optimizer, required
import torch_optimizer as optim
from collections import defaultdict
import itertools as it

import tqdm
import random
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score, log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import timm
from warmup_scheduler import GradualWarmupScheduler
from adamp import AdamP, SGDP

In [ ]:
# class args
class args:
    # ---- factor ---- #
    debug=False
    amp = True
    tta=0
    gpu = '0,1'
    img_size = [512, 512]
    
    batch_size=8#32#128
    weight_decay=1e-6
    n_fold=5
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3
    
    dir_ = f'./saved_models/'
    pt = 'resnet50'#'gluon_seresnext50_32x4d'
    exp_name = f'{img_size}_{pt}_' + 'trash_practice'
    
    warmup_factor=10
    warmup_epo = 5
    cosine_epo = 20
    epochs= cosine_epo+warmup_epo
    #epochs=freeze_epo + warmup_epo + cosine_epo
    
    scheduler = 'CosineAnnealingLR'#'warmupv2'
    
    start_lr = 1e-4 #3e-5
    min_lr= 1e-6
    
    # ---- Dataset ---- #
    
    # factor
    T_max = epochs
    

    # ---- Else ---- #
    num_workers=8
    seed=2021


data_dir = './'
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

##----------------
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed=args.seed)    


In [ ]:
# - util - #
def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

def merge_json():
    train_path = './data/train'
    test_path = './data/test'

    hand_gesture = pd.read_csv('./data/hand_gesture_pose.csv')
    sample_submission = pd.read_csv('./data/sample_submission.csv')
    #
    train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('/')[-1]))
    test_folders  = sorted(glob(test_path + '/*'), key = lambda x : int(x.split('/')[-1]))
    #
    answers = []
    for train_folder in train_folders[1:] :
        json_path = glob(train_folder + '/*.json')[0]
        js = json.load(open(json_path))
        cat = js.get('action')[0]
        cat_name = js.get('action')[1]
        answers.append([train_folder.replace('./data',''),cat, cat_name])

    df = pd.DataFrame(answers, columns = ['folder','pose_id', 'answer_name'])
    df['folder'] = './data' + df['folder']
    
    return df
def load_data(crop=False):
    # train file
    try:
        print('load dataset')
        train_df = pd.read_csv('./data/train.csv')
        test_df = pd.read_csv('./data/test.csv')
    except:
        train_img_path = []
        test_img_path = []
        for (path, dir, files) in os.walk("./data/train"):
            for filename in files:
                #ext = os.path.splitext(filename)[-1]
                if ('ipynb' not in path)&('json' not in filename):
                    train_img_path.append("%s/%s" % (path, filename))

        # test file
        for (path, dir, files) in os.walk("./data/test"):
            for filename in files:
                #ext = os.path.splitext(filename)[-1]
                if ('ipynb' not in path)&('json' not in filename):
                    test_img_path.append("%s/%s" % (path, filename))

        # load all json 
        df = merge_json()

        # train & test
        train_df = pd.DataFrame()
        test_df = pd.DataFrame()

        train_df['path'] = train_img_path
        test_df['path'] = test_img_path

        train_df['folder'] = train_df['path'].apply(lambda x: x.split(x.split('/')[-1])[0][:-1])
        test_df['folder'] = test_df['path'].apply(lambda x: x.split(x.split('/')[-1])[0][:-1])

        # merge target
        train_df = pd.merge(train_df, df[['folder', 'pose_id']], how='left', on='folder')
        train_df = train_df.dropna(axis=0).reset_index(drop=True) # drop 0 folder
        train_df['pose_id'] = train_df['pose_id'].astype(int)

        # encoding label
        le =LabelEncoder()
        train_df['target'] = le.fit_transform(train_df['pose_id'])

        # split fold
        kf = KFold(n_splits=5, random_state=42, shuffle=True)
        train_df['fold'] = -1
        for n_fold, (_,v_idx) in enumerate(kf.split(train_df)):
            train_df.loc[v_idx, 'fold']  = n_fold
        
        # test_df
        sub = pd.read_csv('./data/sample_submission.csv')
        sub['folder'] = './data/test/'+sub['Image_Path'].apply(lambda x: x.split('test')[-1][1:])

        test_df = pd.merge(test_df, sub, how='left', on='folder')
        test_df = test_df.groupby('folder').first().reset_index()[['path','folder']]
        
        train_df.to_csv('./data/train.csv', index=False)
        test_df.to_csv('./data/test.csv', index=False)
        print('Saved train&test csv file!')
    
    
    return train_df, test_df

class Logger(object):
    
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1 ):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass   
def print_args(args, logger=None):
    for k, v in vars(args).items():
        if logger is not None:
            logger.write('{:<16} : {}\n'.format(k, v))
        else:
            print('{:<16} : {}'.format(k, v))
            

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

# models

In [ ]:
# ------------------------
#  dataset
# ------------------------
class MotionDataSet(Dataset):
    
    def __init__(self, data, transform = None, test=False):
        self.data = data # dataframe
        self.test = test # bool
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        
        file_name_input = self.data.iloc[idx]['path']
        
        if self.test: # test
            images = cv2.imread(file_name_input)
            
            if self.transform:
                transformed = self.transform(image=images)
                images = transformed["image"]/255.
            return images
            
            
        else: # train
            images = cv2.imread(file_name_input)
            targets = self.data.iloc[idx]['target']
            if self.transform:
                transformed = self.transform(image=images)
                images = transformed["image"]/255.
        
            
            return images, targets
           
# ------------------------
# transformers(Augmentation)
# ------------------------
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, ImageCompression,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomCrop, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize,
    ChannelShuffle, LongestMaxSize, HueSaturationValue, ISONoise
)

from albumentations.pytorch import ToTensorV2
import albumentations as A
def get_transforms(*, data):
    if data == 'train':
        return A.Compose([
            Resize(args.img_size[0],args.img_size[1]),
            Blur(),
            RandomBrightnessContrast(),
            HueSaturationValue(),
#             A.OneOf([
#                 ChannelShuffle(),
#                 CLAHE()  
#             ], p=0.6),
            
            CoarseDropout(max_holes=4, max_height=16, max_width=16, p=0.5),
            ShiftScaleRotate(rotate_limit=0, p=0.5),
            ImageCompression (quality_lower=40, quality_upper=80, p=0.5),
            GaussNoise(p=0.5),
            ToTensorV2(transpose_mask=False)
        ],p=1.)
    elif data == 'valid':
        return A.Compose([
            Resize(args.img_size[0],args.img_size[1]),
            ToTensorV2(transpose_mask=False)
        ],p=1.)
    


In [ ]:
# ------------------------
#  model
# ------------------------
class SAModels(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(args.pt, pretrained=True)
        modules = list(self.model.children())[:-1]
        self.model = nn.Sequential(*modules)
        if args.pt == 'resnet34':
            self.fc = nn.Linear(512, 157)
        else:
            self.fc = nn.Linear(2048, 157)


    def forward(self, x):
        output = self.model(x)
        output = self.fc(output) # (bs, 1)
        return output


# training

In [ ]:
# ------------------------
#  scheduler
# ------------------------
def get_scheduler(optimizer):
    if args.scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=args.factor, patience=args.patience, 
                                      min_lr = 1e-5, verbose=True, eps=args.eps)
    elif args.scheduler=='CosineAnnealingLR':
        print('scheduler : Cosineannealinglr')
        scheduler = CosineAnnealingLR(optimizer, T_max=args.T_max, eta_min=args.min_lr, last_epoch=-1)
    elif args.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=args.T_0, T_mult=1, eta_min=args.min_lr, last_epoch=-1)
    elif args.scheduler == 'MultiStepLR':
        scheduler = MultiStepLR(optimizer, milestones=args.decay_epoch, gamma= args.factor, verbose=True)
    elif args.scheduler == 'OneCycleLR':
        scheduler = OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                      max_lr=1e-3, epochs=args.epochs, steps_per_epoch=len(train_loader))
        
    elif args.scheduler =='warmupv2':
        print('gradual warmupv2')
        scheduler_cosine=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, args.cosine_epo)
        scheduler_warmup=GradualWarmupSchedulerV2(optimizer, multiplier=args.warmup_factor, total_epoch=args.warmup_epo, after_scheduler=scheduler_cosine)
        scheduler=scheduler_warmup  
        
    else:
        scheduler = None
        print('scheduler is None')
    return scheduler

def do_valid(net, valid_loader, tta=args.tta):

    val_loss = 0
    target_lst = []
    pred_lst = []
    logit = []
    loss_fn = nn.CrossEntropyLoss()

    net.eval()
    start_timer = timer()
    for t, (images, targets) in enumerate(tqdm.tqdm(valid_loader)):
        images  = images.to(device)
        targets  = targets.to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # TTA
                    if tta>1:
                        output=0
                        for t in range(tta):
                            output+= net(images)/tta
                            
                    else:
                        output = net(images)#.squeeze(1)

                    # loss
                    #loss = loss_fn(output, targets)

            else:
                output = net(images)#.squeeze(1)
                loss = loss_fn(output, targets)
            
            #val_loss += loss
            target_lst.append(targets.detach())
            #pred_lst.extend(output.argmax(1).tolist())
            pred_lst.append(output.detach())
    
    target_lst = torch.cat(target_lst, 0)
    pred_lst = torch.cat(pred_lst, 0)
    
    val_mean_loss = loss_fn(pred_lst, target_lst)
    #log_loss(np.eye(target_lst.shape[0])[target_lst], pred_lst) #val_loss / len(valid_loader)
    validation_score = (target_lst== pred_lst.argmax(1)).sum()/target_lst.shape[0]
    #accuracy_score(target_lst, pred_lst.argmax(1))

    return val_mean_loss, validation_score, pred_lst
def do_test(net, test_loader):

    val_loss = 0
    target_lst = []
    pred_lst = []
    logit = []
    loss_fn = nn.CrossEntropyLoss()

    net.eval()
    start_timer = timer()
    for t, images in enumerate(tqdm.tqdm(test_loader)):
        images  = images.to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    output = net(images)#.squeeze(1)

            else:
                output = net(images)#.squeeze(1)
                loss = loss_fn(output, targets)
            
            pred_lst.append(output.detach())
    
    pred_lst = torch.cat(pred_lst, 0)
    
    return pred_lst

def run_train(folds=3):
    out_dir = args.dir_+ f'/fold{args.fold}/{args.exp_name}'
    os.makedirs(out_dir, exist_ok=True)
    log = Logger()
    log.open(out_dir + '/log.train.txt', mode='a')
    print_args(args, log)
    log.write('\n')
    
    # load dataset
    train, test = load_data()    
        
    train_transform = get_transforms(data='train')
    val_transform = get_transforms(data='valid')

    # split fold
    for n_fold in range(5):
        if n_fold != folds:
            print(f'{n_fold} fold pass'+'\n')
            continue
            
        if args.debug:
            train = train.sample(1000).copy()
        
        train_data = train[train['fold']!=n_fold].reset_index(drop=True)
        val_data = train[train['fold']==n_fold].reset_index(drop=True)

        ## dataset ------------------------------------
        train_dataset = MotionDataSet(data = train_data, transform=train_transform)
        valid_dataset = MotionDataSet(data = val_data, transform= tta_transform if args.tta>1  else val_transform)
        trainloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=True, pin_memory=True)
        validloader = DataLoader(dataset=valid_dataset, batch_size=args.batch_size, 
                                 num_workers=8, shuffle=False, pin_memory=True)
        
        ## net ----------------------------------------
        scaler = amp.GradScaler()
        net = SAModels()

        net.to(device)
        if len(args.gpu)>1:
            net = nn.DataParallel(net)

        # ------------------------
        # loss
        # ------------------------
        loss_fn = nn.CrossEntropyLoss()

        # ------------------------
        #  Optimizer
        # ------------------------
        optimizer = AdamP(net.parameters(), lr=args.start_lr, weight_decay=args.weight_decay)
        scheduler = get_scheduler(optimizer)
        
        # ----
        start_timer = timer()
        best_score = 0
        best_loss = 10000
        best_epoch, best_epoch_loss = 0,0
        

        for epoch in range(1, args.epochs+1):
            train_loss = 0
            valid_loss = 0

            target_lst = []
            pred_lst = []
            lr = get_learning_rate(optimizer)
            log.write(f'-------------------')
            log.write(f'{epoch}epoch start')
            log.write(f'-------------------'+'\n')
            log.write(f'learning rate : {lr : .6f}')
            for t, (images, targets) in enumerate(tqdm.tqdm(trainloader)):

                # one iteration update  -------------
                images  = images.to(device)
                targets  = targets.to(device)
                # ------------
                net.train()
                optimizer.zero_grad()

                if args.amp:
                    with amp.autocast():
                        # output
                        output = net(images)#.squeeze(1)

                        # loss
                        loss = loss_fn(output, targets)
                        train_loss += loss


                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                else:
                    # output
                    output = net(images)#.squeeze(1)

                    # loss
                    loss = loss_fn(output, targets)
                    train_loss += loss

                    # update
                    loss.backward()
                    optimizer.step()


                # for calculate f1 score
                target_lst.extend(targets.detach().cpu().numpy())
                pred_lst.extend(output.argmax(1).tolist())


            if scheduler is not None:
                scheduler.step() 
            train_loss = train_loss / len(trainloader)
            train_score = accuracy_score(target_lst, pred_lst)

            # validation
            valid_loss, valid_score, val_preds = do_valid(net, validloader)

            if valid_loss < best_loss:
                best_val_preds = val_preds
                best_loss = valid_loss
                best_epoch = epoch
                print('best LOSS model saved'+'\n')

                torch.save(net.state_dict(), out_dir + f'/{folds}f_{best_epoch}e_{best_loss:.4f}_loss.pth')
                
                
            log.write(f'train loss : {train_loss:.4f}, train ACC score : {train_score : .4f}'+'\n')
            log.write(f'valid loss : {valid_loss:.4f}, valid ACC score : {valid_score : .4f}'+'\n')
            


        log.write(f'best epoch (ACC) : {best_epoch }'+'\n')
        log.write(f'best score : {best_score : .4f}'+'\n')
        log.write(f'best epoch (LOSS) : {best_epoch_loss }'+'\n')
        log.write(f'best score : {best_loss : .4f}'+'\n')
        
        return best_val_preds
    
def run_test(ckpt = ''):
    _, test = load_data()    
        
    val_transform = get_transforms(data='valid')
    
    test_dataset = MotionDataSet(data = test, transform=val_transform, test=True)

    testloader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, 
                             num_workers=8, shuffle=False, pin_memory=True)
    
    net = SAModels()
    net.to(device)
    if len(args.gpu)>1:
        net = nn.DataParallel(net)
    net.load_state_dict(torch.load(ckpt))
    test_preds = do_test(net, testloader)
    
    return test_preds

In [ ]:
"""Training"""
if __name__ == '__main__':
    # model 1
    args.pt = 'resnet50'
    args.img_size = [384, 768]
    preds = run_train(folds=0)
    
    # model 2
    args.img_size = [512, 512]
    preds = run_train(folds=0)
    
    # model 3
    preds = run_train(folds=3)
    
    # model 4
    args.img_size = [384, 768]
    args.pt = 'seresnet50'
    preds = run_train(folds=0)


In [ ]:
"""Inference"""
if __name__ == '__main__':
    test_preds=0
    args.pt = 'resnet50'
    
    # model 1
    args.img_size = [384, 768]
    test_preds += run_test('./pretrained/384_768_2.pth')
    
    # model 2
    args.img_size = [512, 512]
    test_preds += run_test('./pretrained/512_1.pth')
    
    # model 3
    test_preds += run_test('./pretrained/512_2.pth')
    
    # model 4
    args.img_size = [384, 768]
    args.pt = 'seresnet50'
    test_preds += run_test('./pretrained/384_768_1.pth')
    
    # ensemble
    preds= test_preds/4

# submission

In [ ]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.iloc[:,1:] = torch.softmax(preds,1).cpu().numpy()
sub.to_csv('./submission/final_dacon_submission.csv', index=False)